<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB23_M%C3%A9moires_Short_Term_et_Long_Term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB23 : construction d'agents avec mémoires "Short_Term" et "Long_Term"

## Objectifs :
Implémentation de 2 types de mémoire pour les agents IA :
- **Short-term memory** → context window (messages récents)
- **Long-term memory** → knowledge base persistante (Vector DB)

## Temps estimé :
- 20–30 minutes

**Livrables :**
- Notebook montrant comment un agent rappelle les conversations récentes et récupère des faits stockés en mémoire long terme.

## Step 1: Setup (5 min)

Installation des prérequis et dépendances nécessaires.

In [19]:
# Installation des dépendances
!pip install openai langchain chromadb python-dotenv -q
!pip install -q -U langchain-chroma langchain-huggingface langchain-core langchain-openai
!pip install -q datasets
!pip install -q transformers
!pip install -q faiss-cpu
!pip install -q langchain
!pip install -q langchain-community


print("✓ Dépendances installées avec succès")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.3 MB/s eta 0:00:00
✓ Dépendances installées avec succès


### Configuration de l'API OpenAI

Dans Google Colab, vous pouvez définir votre clé API de deux façons :
1. **Méthode sécurisée (recommandée)** : Utiliser `google.colab.userdata`
2. **Méthode alternative** : Définir directement en variable d'environnement

In [14]:
import os
from google.colab import userdata

# Récupérer la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

✓ Clé API OpenAI chargée depuis les secrets Colab


## Step 2: Short-Term Memory (Context Window) (5 min)

La mémoire court-terme consiste à maintenir l'historique de conversation dans le prompt.

### Concept clé :
- Sans contexte → le modèle "oublie"
- Avec historique → il peut se rappeler des informations précédemment données

In [15]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

def chat_with_memory(messages):
    """
    Fonction pour communiquer avec le modèle en gardant l'historique.

    Args:
        messages: Liste de dictionnaires avec 'role' et 'content'

    Returns:
        str: Réponse du modèle
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

print("✓ Fonction chat_with_memory définie")

✓ Fonction chat_with_memory définie


### Expérience 1 : Conversation avec mémoire

Simulons une conversation où l'agent doit se souvenir du nom de l'utilisateur.

In [16]:
# Initialisation une conversation
conversation = [
    {"role": "system", "content": "You are a helpful tutor. Be concise and friendly."},
    {"role": "user", "content": "My name is Stephane."}
]

# Premier échange
print("🗣️ User: My name is Stephane.")
reply1 = chat_with_memory(conversation)
print(f"🤖 Assistant: {reply1}")

# Ajout de la réponse à l'historique
conversation.append({"role": "assistant", "content": reply1})

# Deuxième message - test de mémoire court-terme
conversation.append({"role": "user", "content": "What is my name?"})
print("\n🗣️ User: What is my name?")
reply2 = chat_with_memory(conversation)
print(f"🤖 Assistant: {reply2}")

conversation.append({"role": "assistant", "content": reply2})

🗣️ User: My name is Stephane.
🤖 Assistant: Nice to meet you, Stephane! How can I assist you today?

🗣️ User: What is my name?
🤖 Assistant: Your name is Stephane! How can I help you today?


### Expérience 2 : Sans contexte (oubli)

Demandons le nom du utilisateur **SANS l'historique** pour constater qu'effectivement le modèle oublie.

In [17]:
# Créer une nouvelle conversation SANS contexte précédent
conversation_without_context = [
    {"role": "system", "content": "You are a helpful tutor."},
    {"role": "user", "content": "What is my name?"}
]

print("🗣️ User: What is my name?")
print("\n(Note: Sans contexte précédent dans la conversation)")
reply_without_context = chat_with_memory(conversation_without_context)
print(f"🤖 Assistant: {reply_without_context}")

print("\n" + "="*60)
print("ANALYSE:")
print(f"✓ AVEC contexte: L'agent se souvient du nom (Stephane)")
print(f"✗ SANS contexte: L'agent ne peut pas savoir le nom")
print("="*60)

🗣️ User: What is my name?

(Note: Sans contexte précédent dans la conversation)
🤖 Assistant: I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. How can I assist you today?

ANALYSE:
✓ AVEC contexte: L'agent se souvient du nom (Stephane)
✗ SANS contexte: L'agent ne peut pas savoir le nom


## Step 3: Long-Term Memory (Vector Store) (10 min)

Persistance d'informations y compris après  la fin des sessions de conversation.

### Concept clé :
- Vector DB (Chroma) stocke les embeddings des documents et persiste les informations
- Lors d'un échange, on effectue une recherche sémantique : "agent memory" → récupère les documents pertinents
-  → Les informations sont persistées entre les sessions

In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
#from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Initialiser les embeddings OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("✓ OpenAIEmbeddings initialisé")

✓ OpenAIEmbeddings initialisé


### Créer une base de connaissances

Nous allons stocker plusieurs faits sur les agents IA et LangChain.

In [23]:
# Les 6 dDocuments de notre base de connaissances
knowledge_base = [
    Document(
        page_content="Agentic AI agents use tools and memory to accomplish complex tasks autonomously.",
        metadata={"source": "agentic_ai_basics", "type": "definition"}
    ),
    Document(
        page_content="LangChain is a framework that helps build autonomous agents with memory, tools, and chains.",
        metadata={"source": "langchain_intro", "type": "framework"}
    ),
    Document(
        page_content="RAG (Retrieval-Augmented Generation) improves accuracy by retrieving relevant knowledge before generating responses.",
        metadata={"source": "rag_concept", "type": "technique"}
    ),
    Document(
        page_content="Short-term memory in agents stores recent conversation context in the prompt window.",
        metadata={"source": "agent_memory", "type": "memory_type"}
    ),
    Document(
        page_content="Long-term memory uses vector databases to persistently store and retrieve semantic knowledge.",
        metadata={"source": "agent_memory", "type": "memory_type"}
    ),
    Document(
        page_content="CrewAI enables multi-agent orchestration where multiple agents collaborate to solve problems.",
        metadata={"source": "crewai_intro", "type": "framework"}
    )
]

# Création de la base vectorielle Chroma
db = Chroma.from_documents(
    documents=knowledge_base,
    embedding=embeddings,
    collection_name="hanlab_long_term_memory",
    persist_directory="./chroma_db"
)

print(f"✓ Base vectorielle créée avec {len(knowledge_base)} documents")
print(f"✓ Collection: 'hanlab_long_term_memory'")

✓ Base vectorielle créée avec 6 documents
✓ Collection: 'hanlab_long_term_memory'


### Fonction de rappel (Recall) depuis la base de mémoire long-terme

Récupération des documents pertinents à partir d'une requête.

In [24]:
def recall_from_long_term_memory(query, k=3):
    """
    Récupération des faits pertinents dans la mémoire long-terme.

    Args:
        query:  Chaîne de requête
        k:      Nombre de documents à récupérer

    Returns:
        Liste de documents pertinents
    """
    results = db.similarity_search(query, k=k)
    return results

print("✓ Fonction recall_from_long_term_memory définie")

✓ Fonction recall_from_long_term_memory définie


### Expérience 3 : Récupération de la mémoire long-terme

Tester la récupération sémantique de documents.

In [25]:
# Requête 1
query1 = "How do agents use knowledge?"
print(f"🔍 Query: {query1}")
print("\nRésultats de la mémoire long-terme:")
results1 = recall_from_long_term_memory(query1, k=3)
for i, doc in enumerate(results1, 1):
    print(f"\n  [{i}] {doc.page_content}")
    print(f"      (Source: {doc.metadata['source']})")

print("\n" + "="*60)

🔍 Query: How do agents use knowledge?

Résultats de la mémoire long-terme:

  [1] Agentic AI agents use tools and memory to accomplish complex tasks autonomously.
      (Source: agentic_ai_basics)

  [2] RAG (Retrieval-Augmented Generation) improves accuracy by retrieving relevant knowledge before generating responses.
      (Source: rag_concept)

  [3] Short-term memory in agents stores recent conversation context in the prompt window.
      (Source: agent_memory)



In [26]:
# Requête 2
query2 = "What frameworks help with agent building?"
print(f"🔍 Query: {query2}")
print("\nRésultats de la mémoire long-terme:")
results2 = recall_from_long_term_memory(query2, k=3)
for i, doc in enumerate(results2, 1):
    print(f"\n  [{i}] {doc.page_content}")
    print(f"      (Source: {doc.metadata['source']})")

print("\n" + "="*60)

🔍 Query: What frameworks help with agent building?

Résultats de la mémoire long-terme:

  [1] Agentic AI agents use tools and memory to accomplish complex tasks autonomously.
      (Source: agentic_ai_basics)

  [2] LangChain is a framework that helps build autonomous agents with memory, tools, and chains.
      (Source: langchain_intro)

  [3] CrewAI enables multi-agent orchestration where multiple agents collaborate to solve problems.
      (Source: crewai_intro)



In [27]:
# Requête 3 - Multi-agent orchestration
query3 = "Tell me about multi-agent collaboration"
print(f"🔍 Query: {query3}")
print("\nRésultats de la mémoire long-terme:")
results3 = recall_from_long_term_memory(query3, k=2)
for i, doc in enumerate(results3, 1):
    print(f"\n  [{i}] {doc.page_content}")
    print(f"      (Source: {doc.metadata['source']})")

print("\n" + "="*60)

🔍 Query: Tell me about multi-agent collaboration

Résultats de la mémoire long-terme:

  [1] CrewAI enables multi-agent orchestration where multiple agents collaborate to solve problems.
      (Source: crewai_intro)

  [2] Agentic AI agents use tools and memory to accomplish complex tasks autonomously.
      (Source: agentic_ai_basics)



### Expérience 4 : Ajouter dynamiquement de nouveaux faits

La mémoire long-terme est persistante et extensible.

In [28]:
# Ajouter de nouveaux documents dynamiquement
new_facts = [
    Document(
        page_content="Apache Kafka is a distributed streaming platform for building real-time data pipelines.",
        metadata={"source": "kafka_platform", "type": "technology"}
    ),
    Document(
        page_content="Prompt engineering involves carefully crafting instructions to optimize LLM outputs.",
        metadata={"source": "prompt_engineering", "type": "technique"}
    )
]

# Ajouter les nouveaux documents à la base
db.add_documents(new_facts)

print(f"✓ {len(new_facts)} nouveaux faits ajoutés à la mémoire long-terme")

✓ 2 nouveaux faits ajoutés à la mémoire long-terme


In [47]:
# Vérifier que les nouveaux faits sont récupérables
query_kafka = "Tell me about data streaming platforms"
print(f"🔍 Query: {query_kafka}")
print("\nRésultats (incluant les nouveaux faits):")
results_kafka = recall_from_long_term_memory(query_kafka, k=2)
for i, doc in enumerate(results_kafka, 1):
    print(f"\n  [{i}] {doc.page_content}")
    print(f"      (Source: {doc.metadata['source']})")

🔍 Query: Tell me about data streaming platforms

Résultats (incluant les nouveaux faits):

  [1] Apache Kafka is a distributed streaming platform for building real-time data pipelines.
      (Source: kafka_platform)

  [2] Long-term memory uses vector databases to persistently store and retrieve semantic knowledge.
      (Source: agent_memory)


## Step 4: Association de la mémoire Short-Term et de la mémoire Long-Term (10 min)

Cela consiste à combiner les deux systèmes de mémoire : contexte conversationnel + connaissance persistante.

In [46]:
# 1ère Méthode (déppréciée) :
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains import RetrievalQA
from langchain_openai import ChatOpenAI


# Initialiser le LLM et le retriever
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
retriever = db.as_retriever(search_kwargs={"k": 3})

# Création de la chaîne RAG (Retrieval-Augmented Generation)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

print("✓ Chaîne RetrievalQA créée")
print("✓ Combine short-term (contexte) + long-term (retriever) mémoire")
conversation = [
    {"role":"system","content":"You are a teaching AI agent."},
    {"role":"user","content":"Remember my favorite framework is LangChain."},
    {"role":"assistant","content":"Got it, your favorite framework is LangChain."}
]

# User asks later
conversation.append({"role":"user","content":"What’s my favorite framework and how do agents use memory?"})
print("_______________________________________")
# La 1ère réponse utilise la mémoire court-terme (short-term)
short_term_ans = chat_with_memory(conversation)
print("\nShort-term:", short_term_ans)
print("_______________________________________")
# La seconde réponse est augmentée grace à la mémoire long-terme
print("\nShort+Long-term:", qa_chain.invoke("How do agents use memory?"))
print("_______________________________________")

✓ Chaîne RetrievalQA créée
✓ Combine short-term (contexte) + long-term (retriever) mémoire
_______________________________________

Short-term: Your favorite framework is LangChain. In LangChain, agents can use memory to maintain context and state across interactions. Memory allows agents to remember past interactions, user preferences, and relevant information, enabling them to provide more personalized and context-aware responses.

There are different types of memory implementations in LangChain, such as:

1. **Conversation Memory**: Stores the history of the conversation, allowing the agent to reference previous messages or user inputs.
2. **Long-term Memory**: Keeps track of user-specific information over longer periods, which can help the agent recall important details in future interactions.
3. **Short-term Memory**: Useful for temporary context within a single session, helping the agent respond based on recent interactions without retaining that information long-term.

By utiliz

In [38]:
# 2nde Méthode (LCEL) conseillée désormais :
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# create_retrieval_chain :        Orchestre la récupération (retrieval) + génération
# create_stuff_documents_chain :  Combine les documents récupérés dans un prompt
# ChatPromptTemplate :            Crée un template de prompt structuré
# ChatOpenAI :                    Interface pour appeler GPT (ici GPT-4o-mini)

# Configuration
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
# Configure GPT-4o-mini comme modèle de langue
# temperature=0.7 : Balance entre créativité (1.0) et déterminisme (0.0)
#                   0.7 = réponses modérément créatives mais cohérentes

retriever = db.as_retriever(search_kwargs={"k": 3})
# Convertit db (une base de données vectorielle Chroma/Pinecone/etc.) en retriever
# search_kwargs={"k": 3} : Récupère les 3 documents les plus pertinents
# Le retriever cherchera les chunks sémantiquement proches du query

# Définir le prompt
system_prompt = """Utilisez le contexte fourni pour répondre à la question.
Si vous ne connaissez pas la réponse, dites-le simplement.

Contexte:
{context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Système   : Instructions au modèle (utiliser le contexte, avouer l'ignorance)
# Humain    : Placeholder {input} = la question de l'utilisateur
# Contexte  : Placeholder {context} = sera remplacé par les 3 documents récupérés

# Créer la chaîne moderne
stuff_chain = create_stuff_documents_chain(llm, prompt)
# stuff_chain :
# - Combine tous les documents dans un seul prompt
# - Envoie au LLM : [system_prompt] + [documents] + [question]
# - C'est le pattern "stuff" = on met tout dans le contexte

qa_chain = create_retrieval_chain(retriever, stuff_chain)
# Workflow complet :
# - Reçoit la question utilisateur
# - Appelle retriever → récupère 3 documents
# - Envoie {context} + {input} au stuff_chain
# - Le LLM génère la réponse avec le contexte

print("✓ Chaîne RAG moderne créée avec LCEL")

✓ Chaîne RAG moderne créée avec LCEL


### Expérience 5 : Conversation avec hybridation mémoire

**Scénario :** L'utilisateur indique quel est son framework favori (short-term), puis pose une question impliquant une connaissance long-terme.

In [48]:
# Initialiser une conversation
hybrid_conversation = [
    {
        "role": "system",
        "content": "You are an expert teaching AI agent. You remember user preferences and use knowledge about AI frameworks and techniques."
    },
    {
        "role": "user",
        "content": "My favorite framework is LangChain because it helps build autonomous agents."
    }
]

# Première réponse (short-term memory)
print("🗣️ User: My favorite framework is LangChain because it helps build autonomous agents.")
reply_acknowledge = chat_with_memory(hybrid_conversation)
print(f"🤖 Assistant: {reply_acknowledge}")

hybrid_conversation.append({"role": "assistant", "content": reply_acknowledge})

🗣️ User: My favorite framework is LangChain because it helps build autonomous agents.
🤖 Assistant: That's great to hear! LangChain is indeed a powerful framework for building autonomous agents, especially in the context of natural language processing and interaction. It allows you to create complex workflows by chaining together different components, making it easier to manage state and incorporate various data sources.

If you have any specific questions about LangChain, whether it's about implementation, best practices, or specific features, feel free to ask!


In [49]:
# Question qui demande à la fois short-term et long-term memory
user_query = "What's my favorite framework and how do agents use memory?"
hybrid_conversation.append({"role": "user", "content": user_query})

print(f"\n🗣️ User: {user_query}")
print("\n" + "="*60)
print("RÉPONSE 1: Utilisant SHORT-TERM MEMORY UNIQUEMENT")
print("="*60)

# Réponse sans long-term memory (juste contexte)
short_term_reply = chat_with_memory(hybrid_conversation)
print(f"\n🤖 Assistant:\n{short_term_reply}")


🗣️ User: What's my favorite framework and how do agents use memory?

RÉPONSE 1: Utilisant SHORT-TERM MEMORY UNIQUEMENT

🤖 Assistant:
Your favorite framework is LangChain. 

In LangChain, agents can use memory to retain information across interactions, enabling them to act more intelligently and contextually. Memory allows agents to remember facts, previous interactions, user preferences, and other relevant data, which helps create a more coherent and personalized experience. 

There are different types of memory that agents can utilize in LangChain:

1. **Short-Term Memory**: This allows the agent to remember information only for the duration of a single session. It's useful for maintaining context during an ongoing conversation.

2. **Long-Term Memory**: This type of memory enables agents to store information persistently across sessions. It can be useful for remembering user preferences, previous conversations, and important facts that should influence future interactions.

3. **Cus

In [51]:
print("\n" + "="*60)
print("RÉPONSE 2: Utilisant SHORT-TERM + LONG-TERM MEMORY (RAG)")
print("="*60)

# Réponse avec long-term memory retrieval
# Dépprécié : qa_result = qa_chain({"query": user_query})
qa_result = qa_chain.invoke({"query": user_query})

long_term_reply = qa_result["result"]

print(f"\n🤖 Assistant (enrichie avec RAG):\n{long_term_reply}")

# Afficher les sources récupérées
print("\n📚 Documents récupérés de la mémoire long-terme:")
for i, doc in enumerate(qa_result.get("source_documents", []), 1):
    print(f"  [{i}] {doc.page_content[:80]}...")
    print(f"      (Source: {doc.metadata['source']})")


RÉPONSE 2: Utilisant SHORT-TERM + LONG-TERM MEMORY (RAG)

🤖 Assistant (enrichie avec RAG):
I don't know what your favorite framework is. However, agents in LangChain use memory to store recent conversation context in the prompt window, which helps them accomplish complex tasks autonomously.

📚 Documents récupérés de la mémoire long-terme:
  [1] LangChain is a framework that helps build autonomous agents with memory, tools, ...
      (Source: langchain_intro)
  [2] Agentic AI agents use tools and memory to accomplish complex tasks autonomously....
      (Source: agentic_ai_basics)
  [3] Short-term memory in agents stores recent conversation context in the prompt win...
      (Source: agent_memory)


## Step 5: Reflection & Experiment (15–20 min)

Expériences avancées pour comprendre les limites et avantages.

### Expérience 6 : Oubli avec réinitialisation de conversation (short-term)

Démontrer que la mémoire court-terme se perd lors du reset.

In [52]:
print("SCENARIO: Réinitialisation de conversation\n")
print("="*60)

# Conversation 1 : L'agent apprend une info
conv_session1 = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I love Apache Kafka for real-time data processing."}
]
reply_session1 = chat_with_memory(conv_session1)
print(f"Session 1 - User: I love Apache Kafka for real-time data processing.")
print(f"Session 1 - Assistant: {reply_session1}\n")

# Conversation 2 : NOUVELLE CONVERSATION (conversation réinitialisée)
conv_session2 = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What do I love for data processing?"}
]
reply_session2 = chat_with_memory(conv_session2)
print(f"Session 2 - User: What do I love for data processing?")
print(f"Session 2 - Assistant: {reply_session2}")
print(f"\n❌ SHORT-TERM MEMORY: L'agent a OUBLIÉ (conversation réinitialisée)")
print("="*60)

SCENARIO: Réinitialisation de conversation

Session 1 - User: I love Apache Kafka for real-time data processing.
Session 1 - Assistant: That's great to hear! Apache Kafka is a powerful distributed event streaming platform that excels in real-time data processing. It allows you to build robust data pipelines and stream applications. Some of the key features and benefits of Kafka include:

1. **Scalability**: Kafka can handle large volumes of data, making it suitable for high-throughput applications.

2. **Fault Tolerance**: Kafka replicates data across multiple brokers, ensuring that your data is safe even in the event of hardware failures.

3. **Durability**: Messages in Kafka are persisted to disk, which means you can replay them if needed.

4. **High Performance**: Kafka is designed for high-throughput and low-latency, making it ideal for real-time analytics and data processing.

5. **Decoupling of Systems**: Kafka allows producers and consumers to operate independently, enabling a m

### Expérience 7 : Persistance avec long-term memory

Même après réinitialisation, la long-term memory persiste.

In [54]:
print("SCENARIO: Requête après réinitialisation avec LONG-TERM MEMORY\n")
print("="*60)

# Même question, mais avec RAG
query_persistent = "What technologies are mentioned for real-time processing?"
print(f"Query: {query_persistent}")

# Utiliser le RAG chain
qa_persistent = qa_chain.invoke({"query": query_persistent})
result_persistent = qa_persistent["result"]

print(f"\n🤖 Assistant (with long-term memory):\n{result_persistent}")
print(f"\n✓ LONG-TERM MEMORY: Les connaissances PERSISTENT même après reset")
print("="*60)

SCENARIO: Requête après réinitialisation avec LONG-TERM MEMORY

Query: What technologies are mentioned for real-time processing?

🤖 Assistant (with long-term memory):
I don't know.

✓ LONG-TERM MEMORY: Les connaissances PERSISTENT même après reset


### Expérience 8 : Tester hallucination vs retrieval

Comparer les réponses avec et sans contexte de retrieval.

In [ ]:
# Question qui pourrait causer une hallucination
hallucination_query = "What is the relationship between multi-agent systems and real-time data processing?"

print(f"🔍 Query: {hallucination_query}\n")
print("="*60)
print("RÉPONSE 1: Sans retrieval (risque de hallucination)")
print("="*60)

# Réponse directe (sans RAG)
conv_no_rag = [
    {"role": "system", "content": "You are an AI expert."},
    {"role": "user", "content": hallucination_query}
]
reply_no_rag = chat_with_memory(conv_no_rag)
print(f"\n{reply_no_rag}")

print("\n" + "="*60)
print("RÉPONSE 2: Avec retrieval (grounding knowledge)")
print("="*60)

# Réponse avec RAG
qa_grounded = qa_chain({"query": hallucination_query})
reply_rag = qa_grounded["result"]
print(f"\n{reply_rag}")
print(f"\nDocuments utilisés comme source:")
for i, doc in enumerate(qa_grounded["source_documents"], 1):
    print(f"  [{i}] {doc.page_content}")

### Expérience 9 : Ajouter des faits dynamiquement et tester

Simuler un agent qui apprend continuellement.

In [ ]:
print("SCENARIO: Apprentissage continu (agent learns new facts)\n")
print("="*60)

# Ajouter un nouveau fait important
fact_to_learn = Document(
    page_content="Function calling enables AI agents to interact with external APIs and tools to perform real-world actions.",
    metadata={"source": "agent_capabilities", "type": "technique"}
)

db.add_documents([fact_to_learn])
print("✓ Nouveau fait ajouté: 'Function calling enables AI agents...'")

# Tester la récupération
test_query = "How can agents interact with external systems?"
print(f"\n🔍 Query: {test_query}")

qa_updated = qa_chain({"query": test_query})
print(f"\n🤖 Assistant (with updated knowledge):\n{qa_updated['result']}")

print(f"\n✓ Agent a accès au nouveau fait appris")

## Summary & Key Insights

### Récapitulatif de ce que vous avez appris :

In [ ]:
summary = """\n╔════════════════════════════════════════════════════════════════════╗
║         SHORT-TERM vs LONG-TERM MEMORY IN AGENTIC AI              ║
╚════════════════════════════════════════════════════════════════════╝

📌 SHORT-TERM MEMORY (Context Window)
─────────────────────────────────────
✓ Stocke l'historique récent de conversation
✓ Permet au modèle de référencer les messages précédents
✓ Limité par la taille de la context window (ex: 4K, 8K, 128K tokens)
✗ Se réinitialise à chaque nouvelle conversation
✗ Ne persiste pas entre les sessions

Cas d'usage:
• Conversations interactives
• Contexte immédiat et références
• Dialogue naturel

---

💾 LONG-TERM MEMORY (Vector Database)
──────────────────────────────────────
✓ Stocke les connaissances de façon persistante
✓ Récupération sémantique (recherche par similarité)
✓ Scalable à de grands corpus de connaissances
✓ Persiste entre les sessions
✓ Améliore accuracy et réduit hallucination (RAG)
✗ Nécessite une setup supplémentaire
✗ Coût en appels embeddings

Cas d'usage:
• Knowledge bases persistantes
• Fact retrieval
• Context augmentation (RAG)
• Agents autonomes à long terme

---

🔗 HYBRID APPROACH (Short-term + Long-term)
────────────────────────────────────────────
✓ Combine conversational context + persistent knowledge
✓ Meilleure compréhension du contexte utilisateur
✓ Accès à des faits précis stockés
✓ Réduction des hallucinations
✓ Agents plus intelligents et contextuels

Flux:
1. Contexte court-terme: Lire l'historique conversation
2. Récupération long-terme: Chercher docs pertinents via RAG
3. Fusion: Combiner contexte + docs pour meilleure réponse

---

🚀 NEXT STEPS
─────────────
1. Essayez avec vos propres documents (PDFs, web pages)
2. Tunerez embedding model et retriever (top_k, distance threshold)
3. Implémentez un agent loop complet (ReAct pattern)
4. Explorez multi-agent orchestration (CrewAI)
5. Comparez différentes Vector DBs: Chroma vs Pinecone vs Weaviate
"""
print(summary)

## 🎓 Challenges & Questions

Essayez ces variations pour approfondir votre compréhension :

1. **Challenge 1:** Ajoutez 5 nouveaux faits à la base de connaissances. Testez les requêtes complexes qui les combinent.

2. **Challenge 2:** Créez une conversation multi-tour où l'agent doit:
   - Se rappeler le nom de l'utilisateur (short-term)
   - Récupérer des faits sur les frameworks (long-term)
   - Générer une réponse personnalisée

3. **Challenge 3:** Comparez 2 différentes requêtes:
   - L'une qui demande information dans la base (retrievable)
   - L'une qui demande information hors de la base (hallucination risk)

4. **Challenge 4:** Implémentez un feedback loop:
   - L'agent génère une réponse
   - Vous évaluez la confiance (hallucination ou pas)
   - Ajoutez des documents pour améliorer futures requêtes

5. **Challenge 5:** Explorez `similarity_search_with_score()` pour voir les distances d'embedding

## 📖 Ressources supplémentaires

- [LangChain Documentation](https://python.langchain.com/)
- [Chroma Vector Database](https://www.trychroma.com/)
- [OpenAI Embeddings](https://platform.openai.com/docs/models/embeddings)
- [RAG Pattern](https://python.langchain.com/docs/use_cases/question_answering/)
- [CrewAI for Multi-Agent](https://crewai.io/)

In [ ]:
print("\n🎉 Lab terminé!")
print("✅ Vous avez appris:")
print("   • Comment construire une mémoire court-terme (context window)")
print("   • Comment construire une mémoire long-terme (Vector DB)")
print("   • Comment combiner les deux pour des agents intelligents")
print("   • Comment éviter les hallucinations avec RAG")
print("\n📝 Sauvegardez ce notebook pour références futures!")